In [5]:
import requests

# BLS blocks requests without a proper User-Agent
headers = {
    "User-Agent": "YourName (nahomkenna14@outlook.com)"
}

url_bls = "https://download.bls.gov/pub/time.series/pr/pr.data.0.Current"
bls_response = requests.get(url_bls, headers=headers)

with open("pr.data.0.Current", "wb") as f:
    f.write(bls_response.content)

print("✅ pr.data.0.Current downloaded successfully with headers")


✅ pr.data.0.Current downloaded successfully with headers


In [6]:
import pandas as pd
import json

# Load time series from Part 1
df_timeseries = pd.read_csv("pr.data.0.Current", sep="\t")
df_timeseries.columns = df_timeseries.columns.str.strip()
df_timeseries.head()


,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


In [7]:
# Download your previously uploaded JSON from S3
url_json = "https://rearc-bls-nahom.s3.amazonaws.com/datausa/population.json"
json_response = requests.get(url_json)

with open("population.json", "wb") as f:
    f.write(json_response.content)

print("✅ population.json downloaded")


✅ population.json downloaded


In [8]:
# Load population.json from Part 2
with open("population.json") as f:
    data = json.load(f)

# Normalize nested JSON structure into flat table
df_population = pd.json_normalize(data["data"])
df_population.columns = df_population.columns.str.strip()

# Preview the data
df_population.head()


,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2023,2023,332387540,united-states
1,01000US,United States,2022,2022,331097593,united-states
2,01000US,United States,2021,2021,329725481,united-states
3,01000US,United States,2020,2020,326569308,united-states
4,01000US,United States,2019,2019,324697795,united-states


In [10]:
# 🔄 Convert 'Year' and 'Population' to numeric types
df_population['Year'] = pd.to_numeric(df_population['Year'], errors='coerce')
df_population['Population'] = pd.to_numeric(df_population['Population'], errors='coerce')

# 📆 Filter for years 2013–2018
df_pop_filtered = df_population[df_population['Year'].between(2013, 2018)]

# 📊 Compute mean and standard deviation
mean_pop = df_pop_filtered['Population'].mean()
std_pop = df_pop_filtered['Population'].std()

# 🖨️ Print results
print("📊 Mean U.S. Population (2013–2018):", round(mean_pop))
print("📉 Standard Deviation:", round(std_pop))


📊 Mean U.S. Population (2013–2018): 317437383
📉 Standard Deviation: 4257090


In [11]:
# 🧹 Clean the 'value' column
df_timeseries['value'] = pd.to_numeric(df_timeseries['value'], errors='coerce')

# Drop rows where 'value' is missing
df_timeseries = df_timeseries.dropna(subset=['value'])

# 📊 Group by series_id and year, summing up all quarterly values
df_grouped = df_timeseries.groupby(['series_id', 'year'])['value'].sum().reset_index()

# 🏆 For each series_id, get the year with the highest total value
df_best_years = df_grouped.loc[df_grouped.groupby('series_id')['value'].idxmax()].reset_index(drop=True)

# 🖨️ Show the result
df_best_years.head()


,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,705.895
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.400


In [12]:
# 🔍 Step 1: Filter the time-series data for PRS30006032 and Q01
df_filtered_value = df_timeseries[
    (df_timeseries['series_id'] == 'PRS30006032') &
    (df_timeseries['period'] == 'Q01')
][['series_id', 'year', 'period', 'value']]

# 🧼 Step 2: Ensure 'year' in both DataFrames is numeric
df_filtered_value['year'] = pd.to_numeric(df_filtered_value['year'], errors='coerce')
df_population['Year'] = pd.to_numeric(df_population['Year'], errors='coerce')
df_population['Population'] = pd.to_numeric(df_population['Population'], errors='coerce')

# 🔁 Step 3: Prepare population DataFrame and rename 'Year' to match
df_pop_subset = df_population[['Year', 'Population']].rename(columns={'Year': 'year'})

# 🔗 Step 4: Merge on 'year'
df_joined = pd.merge(df_filtered_value, df_pop_subset, on='year', how='inner')

# ✅ Final output
df_joined.head()


,series_id,year,period,value,Population


In [13]:
df_filtered_value['year'].unique()


array([], dtype=int64)

In [14]:
array([2018])



NameError: name 'array' is not defined

In [15]:
df_pop_subset['year'].unique()


array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013])

In [16]:
df_timeseries['series_id'].unique()


array(['PRS30006011      ', 'PRS30006012      ', 'PRS30006013      ',
       'PRS30006021      ', 'PRS30006022      ', 'PRS30006023      ',
       'PRS30006031      ', 'PRS30006032      ', 'PRS30006033      ',
       'PRS30006061      ', 'PRS30006062      ', 'PRS30006063      ',
       'PRS30006081      ', 'PRS30006082      ', 'PRS30006083      ',
       'PRS30006091      ', 'PRS30006092      ', 'PRS30006093      ',
       'PRS30006101      ', 'PRS30006102      ', 'PRS30006103      ',
       'PRS30006111      ', 'PRS30006112      ', 'PRS30006113      ',
       'PRS30006131      ', 'PRS30006132      ', 'PRS30006133      ',
       'PRS30006151      ', 'PRS30006152      ', 'PRS30006153      ',
       'PRS30006161      ', 'PRS30006162      ', 'PRS30006163      ',
       'PRS30006171      ', 'PRS30006172      ', 'PRS30006173      ',
       'PRS30006211      ', 'PRS30006212      ', 'PRS30006213      ',
       'PRS30006221      ', 'PRS30006222      ', 'PRS30006223      ',
       'PRS30006231 

In [17]:
df_timeseries[df_timeseries['series_id'] == 'PRS30006032']['period'].unique()


array([], dtype=object)

In [18]:
df_timeseries[df_timeseries['series_id'] == 'PRS30006032']



,series_id,year,period,value,footnote_codes


In [20]:
df_filtered_value = df_timeseries[
    (df_timeseries['series_id'] == 'PRS30006011') &
    (df_timeseries['period'] == 'Q01')
][['series_id', 'year', 'period', 'value']]


In [21]:
# Use a confirmed valid series_id and period
df_filtered_value = df_timeseries[
    (df_timeseries['series_id'] == 'PRS30006011') &
    (df_timeseries['period'] == 'Q01')
][['series_id', 'year', 'period', 'value']]

# Make sure types match
df_filtered_value['year'] = df_filtered_value['year'].astype(int)
df_population['Year'] = pd.to_numeric(df_population['Year'], errors='coerce')
df_population['Population'] = pd.to_numeric(df_population['Population'], errors='coerce')

# Rename and join
df_pop_subset = df_population[['Year', 'Population']].rename(columns={'Year': 'year'})
df_joined = pd.merge(df_filtered_value, df_pop_subset, on='year', how='inner')

# Show result
df_joined.head()


,series_id,year,period,value,Population


In [22]:
df_timeseries[df_timeseries['series_id'] == 'PRS30006011']['period'].unique()


array([], dtype=object)

In [25]:
import requests

# BLS blocks requests without a proper User-Agent
headers = {
    "User-Agent": "YourName (nahomkenna14@outlook.com)"
}

url_bls = "https://download.bls.gov/pub/time.series/pr/pr.data.0.Current"
bls_response = requests.get(url_bls, headers=headers)

with open("pr.data.0.Current", "wb") as f:
    f.write(bls_response.content)

print("✅ pr.data.0.Current downloaded successfully with headers")

✅ pr.data.0.Current downloaded successfully with headers


In [26]:
import pandas as pd
import json

# Load time series from Part 1
df_timeseries = pd.read_csv("pr.data.0.Current", sep="\t")
df_timeseries.columns = df_timeseries.columns.str.strip()
df_timeseries.head()

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


In [27]:
df_timeseries[df_timeseries['series_id'] == 'PRS30006011']['period'].unique()


array([], dtype=object)

In [28]:
df_timeseries.loc[df_timeseries['series_id'] == 'PRS30006011', 'period'].unique()


array([], dtype=object)

In [29]:
# Strip leading/trailing whitespace from all string columns
df_timeseries = df_timeseries.applymap(lambda x: x.strip() if isinstance(x, str) else x)


C:\Users\Owner\AppData\Local\Temp\ipykernel_6688\561936647.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_timeseries = df_timeseries.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [30]:
import requests

# BLS blocks requests without a proper User-Agent
headers = {
    "User-Agent": "YourName (nahomkenna14@outlook.com)"
}

url_bls = "https://download.bls.gov/pub/time.series/pr/pr.data.0.Current"
bls_response = requests.get(url_bls, headers=headers)

with open("pr.data.0.Current", "wb") as f:
    f.write(bls_response.content)

print("✅ pr.data.0.Current downloaded successfully with headers")

✅ pr.data.0.Current downloaded successfully with headers


In [31]:
import pandas as pd
import json

# Load time series from Part 1
df_timeseries = pd.read_csv("pr.data.0.Current", sep="\t")
df_timeseries.columns = df_timeseries.columns.str.strip()
df_timeseries.head()

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


In [32]:
# Strip leading/trailing whitespace from all string columns
df_timeseries = df_timeseries.applymap(lambda x: x.strip() if isinstance(x, str) else x)


C:\Users\Owner\AppData\Local\Temp\ipykernel_6688\561936647.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_timeseries = df_timeseries.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [33]:
# Strip leading/trailing whitespace from all string columns
df_timeseries = df_timeseries.map(lambda x: x.strip() if isinstance(x, str) else x)


In [34]:
df_timeseries.loc[df_timeseries['series_id'] == 'PRS30006011', 'period'].unique()

array(['Q01', 'Q02', 'Q03', 'Q04', 'Q05'], dtype=object)

In [35]:
df_timeseries.loc[df_timeseries['series_id'] == 'PRS30006032', 'period'].unique()

array(['Q01', 'Q02', 'Q03', 'Q04', 'Q05'], dtype=object)

In [36]:
# Step 1: Filter for the target series_id and period
df_filtered_value = df_timeseries[
    (df_timeseries['series_id'] == 'PRS30006032') &
    (df_timeseries['period'] == 'Q01')
][['series_id', 'year', 'period', 'value']]

# Step 2: Clean types
df_filtered_value['year'] = pd.to_numeric(df_filtered_value['year'], errors='coerce')
df_population['Year'] = pd.to_numeric(df_population['Year'], errors='coerce')
df_population['Population'] = pd.to_numeric(df_population['Population'], errors='coerce')

# Step 3: Join with population data
df_pop_subset = df_population[['Year', 'Population']].rename(columns={'Year': 'year'})
df_joined = pd.merge(df_filtered_value, df_pop_subset, on='year', how='inner')

# ✅ Final Output
df_joined.head()


,series_id,year,period,value,Population
0,PRS30006032,2013,Q01,0.5,311536594
1,PRS30006032,2014,Q01,-0.1,314107084
2,PRS30006032,2015,Q01,-1.7,316515021
3,PRS30006032,2016,Q01,-1.4,318558162
4,PRS30006032,2017,Q01,0.9,321004407
